# Get started with Azure Cosmos DB using the .NET SDK for SQL API

In this notebook, we'll learn how to do operations against the Azure Cosmos DB SQL API using the .NET SDK client library. We'll use an IOT scenario of vehicle telemetry to illustrate how to create resources, insert data, query and visualize our data. 

<br>To run this notebook, be sure to select the **CSharp** kernel in the command bar, so we can get all the language support features we need.

### Import NuGet packages
First, we'll import the NuGet packages we need in this notebook. Use the syntax: ``#r "nuget: PackageName, OptionalVersionNumber"``. 

In [1]:
#r "nuget: Bogus"
#r "nuget: System.Linq"
#r "nuget: System.Linq.Expressions"

### Create database and containers
Next, we'll create a new database and container to hold our data. Note when we create the container, we select a value for the partition key: we'll partition our data by Vehicle Identification Number (VIN), as it has a high cardinality (important for workloads during a lot of writes, e.g. IOT workloads) and evenly distributes the request and storage volume. Choosing a good partition key is "key" to getting good scale and performance from Azure Cosmos DB, so it's important we follow the [best practices](https://docs.microsoft.com/azure/cosmos-db/partitioning-overview)!

The type of container this will create depends on the capacity mode of your Cosmos DB account:
- On a provisioned throughput account, this will create a container provisioned at 400 RU/s, the minimum RU/s for any provisioned throughput container.
- On a serverless account, this will create a serverless container.


In [1]:
using System;
using Microsoft.Azure.Cosmos;
using System.Collections;

// Initialize a new instance of CosmosClient using the built-in account endpoint and key parameters
CosmosClient cosmosClient = new CosmosClient(Cosmos.Endpoint, Cosmos.Key);

// Create a new database and a new container
Microsoft.Azure.Cosmos.Database database = await cosmosClient.CreateDatabaseIfNotExistsAsync("CosmosIOTDemo");
Container container = await database.CreateContainerIfNotExistsAsync("VehicleData", "/vin");

Display.AsMarkdown(@"
Created database CosmosIOTDemo and container VehicleData. You can see these new resources by refreshing your resource pane under the Data section.
");


Created database CosmosIOTDemo and container VehicleData. You can see these new resources by refreshing your resource pane under the Data section.


### Create C# classes to represent our TelemetryEvent
We can create classes to represent the items we will be inserting into our container. In this sample, we'll be working with vehicle telemetry data that contains sensor information.

In [3]:
using System;

class TelemetryEvent
{
    public string id { get; set; }
    
    public string vin { get; set; }

    public string eventName { get; set; }

    public string vehicleType { get; set; }
        
    public string fuelType {get; set; }    

    public double s1 { get; set; }

    public double s2 { get; set; }

    public double s3 { get; set; }

    public DateTime timestamp { get; set; }

    public string date { get; set; }

    public string region { get; set; }

}

### Insert data into our container
First, we'll write a helper method we can call that randomly generates a set number of events that we will insert into our container. We'll use the [Bogus](https://github.com/bchavez/Bogus) library for .NET to easily generate sample data.

In [4]:
using System.Linq;
using System.Collections.Generic;
using System.Linq.Expressions;
using System;
using Bogus;

static List<TelemetryEvent> GenerateTelemetryEvents(int numberOfEventsToGenerate)
{
    var eventTypes = new string[] { "Harsh_break", "Airbag_deploy", "Check_engine_light" };

    var faker = new Faker();
    
    var vinList = Enumerable.Range(1, 10) //Generate 10 different vehicle ids
    .Select(_ => _.ToString())
    .ToList();
    
    var telemetryEvent = new Faker<TelemetryEvent>()
    .StrictMode(true)
    //Generate telemetry event
    .RuleFor(o => o.id, f => Guid.NewGuid().ToString())
    .RuleFor(o => o.vin, f => f.PickRandom(vinList))
    .RuleFor(o => o.eventName, f => f.PickRandom(eventTypes))
    .RuleFor(o => o.vehicleType, f => f.Vehicle.Type())
    .RuleFor(o => o.fuelType, f => f.Vehicle.Fuel())
    .RuleFor(o => o.timestamp, f => DateTime.Now) // just today's date
    .RuleFor(o => o.date, (f, m) => $"{m.timestamp.ToString("yyyy-MM-dd")}")
    .RuleFor(o => o.s1, f => f.Random.Double(1, 2000))
    .RuleFor(o => o.s2, f => f.Random.Double(1, 2000))
    .RuleFor(o => o.s3, f => f.Random.Double(1, 2000))
    .RuleFor(o => o.region, f => f.Address.Country());
    
    var events = telemetryEvent.Generate(numberOfEventsToGenerate);
       
    return events;

}


Next, let's generate 100 events and write them to our container.

In [2]:
var telemetryEvents = GenerateTelemetryEvents(100);

foreach (TelemetryEvent telemetryEvent in telemetryEvents)
{
    var itemResponse = await container.CreateItemAsync<TelemetryEvent>(telemetryEvent);
}

Display.AsMarkdown(@"
Created 100 items in VehicleData container. 
");




Created 100 items in VehicleData container. 


### Query for items
We can run a [SQL query](https://docs.microsoft.com/azure/cosmos-db/sql-query-getting-started) against our container. Azure Cosmos DB returns results in pages, so we use the pattern of calling the [ReadNextAsync](https://docs.microsoft.com/dotnet/api/microsoft.azure.cosmos.feediterator.readnextasync?view=azure-dotnet) method on a [FeedIterator](https://docs.microsoft.com/dotnet/api/microsoft.azure.cosmos.container.getitemqueryiterator?view=azure-dotnet#Microsoft_Azure_Cosmos_Container_GetItemQueryIterator__1_Microsoft_Azure_Cosmos_QueryDefinition_System_String_Microsoft_Azure_Cosmos_QueryRequestOptions_). 

The query ``SELECT * FROM c WHERE c.vin = '8'`` will return all documents for the specified ``vin``. Putting the partition key of ``vin`` in the WHERE clause ensures that we can run an efficient query. 

In [6]:
QueryDefinition queryDefinition = new QueryDefinition("SELECT TOP 5 * FROM c WHERE c.vin = @vin")
                                      .WithParameter("@vin", "8");

FeedIterator<TelemetryEvent> queryResultSetIterator = container.GetItemQueryIterator<TelemetryEvent>(queryDefinition);

List<TelemetryEvent> telemetryEvents = new List<TelemetryEvent>();

while (queryResultSetIterator.HasMoreResults)
{
    FeedResponse<TelemetryEvent> currentResultSet = await queryResultSetIterator.ReadNextAsync();
    foreach (TelemetryEvent telemetryEvent in currentResultSet)
    {
        telemetryEvents.Add(telemetryEvent);
    }
}

telemetryEvents

System.Collections.Generic.List`1[Submission#6+TelemetryEvent]

### Next steps
- See [samples](https://github.com/Azure/azure-cosmos-dotnet-v3/tree/master/Microsoft.Azure.Cosmos.Samples) for the [Azure Cosmos DB .NET SDK](https://www.nuget.org/packages/Microsoft.Azure.Cosmos/)
- Learn how to choose between [provisioned throughput and serverless](https://docs.microsoft.com/en-us/azure/cosmos-db/throughput-serverless)
- Learn about how to choose the right [throughput (RU/s)](https://docs.microsoft.com/azure/cosmos-db/estimate-ru-with-capacity-planner) when using provisioned throughput
- Learn about how to [choose a good partition key](https://docs.microsoft.com/azure/cosmos-db/partitioning-overview) and [model your data](https://docs.microsoft.com/azure/cosmos-db/how-to-model-partition-example)